In [4]:
import folium
import re

In [5]:
import pandas as pd

rest = pd.read_csv(r"C:\Users\JOLA\Desktop\Restaurant\archive\restaurants.csv")
menu = pd.read_csv(r'C:\Users\JOLA\Desktop\Restaurant\archive\restaurant-menus.csv')

In [6]:
# wyświetlenie pięciu pierwszych wierszy tabeli
menu.head()


,restaurant_id,category,name,description,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD


In [7]:
# wyświetlenie informacji o tabeli
menu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375211 entries, 0 to 3375210
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   restaurant_id  int64 
 1   category       object
 2   name           object
 3   description    object
 4   price          object
dtypes: int64(1), object(4)
memory usage: 128.8+ MB


In [36]:
menu_copy = menu.copy()
rest_copy = rest.copy()

In [37]:
menu_copy = menu_copy.dropna()
rest_copy = rest_copy.dropna()

In [23]:
# wyświetlenie wartości unikalnych dla kolumny 'category'
print(menu_copy['name'].unique())

['Extra Large Meat Lovers' 'Extra Large Supreme' 'Extra Large Pepperoni'
 ... 'Panda Plate' 'Tiger Plate' 'Lion Plate']


In [26]:
##czego jest najwięej w ofertach restauracji:
count_products = menu_copy['name'].value_counts()
count_products[:20]

Tuna                             2813
Roast Beef                       2799
Buffalo Chicken                  2787
Steak &amp; Cheese               2711
Caesar Salad                     2671
B.L.T.                           2624
Spicy Italian                    2622
Meatball Marinara                2606
Black Forest Ham                 2575
Chicken &amp; Bacon Ranch        2572
Rotisserie-Style Chicken         2549
Cold Cut Combo®                  2542
Baja Chicken &amp; Bacon         2508
Steak "Cali Fresh"               2497
Sweet Onion Chicken Teriyaki     2494
Turkey "Cali Fresh"              2450
Subway Club®                     2439
Oven Roasted Turkey &amp; Ham    2401
Baja Turkey Avocado              2365
Supreme Meats                    2347
Name: name, dtype: int64

In [28]:
##usunięcie USD z cen
menu_copy['price'] = menu_copy['price'].replace(' USD', '')

In [29]:
##zmiana na numeric
menu_copy['price'] = pd.to_numeric(menu_copy['price'], errors='coerce')

In [30]:
## wyświetlenie informacji o restauracjach, w których można zamówić dany produkt
bottled_water =menu_copy[menu_copy['name'] == 'Bottled Water']['restaurant_id']

In [31]:
bottled_water = menu_copy.loc[menu_copy['name'] == 'Bottled Water', ['restaurant_id', 'price']]
bottled_water


,restaurant_id,price
448,9,1.99
637,14,2.29
1791,36,0.00
10821,166,2.29
13362,198,2.63
...,...,...
3360639,39999,2.59
3366308,40102,2.46
3370120,40159,2.49
3371087,40171,1.00


In [32]:
##średnia cena wody:
mean_price = round(bottled_water.price.mean(),2)
mean_price

2.4

In [33]:
count = menu_copy[(menu_copy['name'] == 'Bottled Water') & (menu_copy['price'] > 2.3)]['name'].count()
count

852

In [34]:
count2 = menu_copy[(menu_copy['name'] == 'Bottled Water') & (menu_copy['price'] < 2.3)]['name'].count()
count2

525

In [66]:
##dodanie do tabeli menu lng i lat z tabeli restaurant
menu_rest = pd.merge(menu_copy, rest_copy, left_on='restaurant_id', right_on='id', how='left')
menu_rest

,restaurant_id,category_x,name_x,description,price,id,position,name_y,score,ratings,category_y,price_range,full_address,zip_code,lat,lng
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466909,40227,Chef's Special,Dragon and Tiger Fight,HALF GENERAL TSAO'S CHICKEN AND HALF SHRIMP WI...,13.95,40227.0,50.0,Beijing Wok,4.4,254.0,"Chinese, Asian, Asian Fusion, Family Friendly,...",$,"8106 Brodie Ln, Austin, TX, 78749",78749,30.20221,-97.838689
2466910,40227,Chef's Special,China Town,"CHICKEN,BEEF AND SHRIMP STIR FRIED WITH MIXED ...",13.95,40227.0,50.0,Beijing Wok,4.4,254.0,"Chinese, Asian, Asian Fusion, Family Friendly,...",$,"8106 Brodie Ln, Austin, TX, 78749",78749,30.20221,-97.838689
2466911,40227,Kid’s Mains,Panda Plate,Chicken wings two pieces and golden chicken fi...,8.55,40227.0,50.0,Beijing Wok,4.4,254.0,"Chinese, Asian, Asian Fusion, Family Friendly,...",$,"8106 Brodie Ln, Austin, TX, 78749",78749,30.20221,-97.838689
2466912,40227,Kid’s Mains,Tiger Plate,Crab puff two pieces and egg rolls two pieces....,8.55,40227.0,50.0,Beijing Wok,4.4,254.0,"Chinese, Asian, Asian Fusion, Family Friendly,...",$,"8106 Brodie Ln, Austin, TX, 78749",78749,30.20221,-97.838689


In [67]:
##max
max_price_water = menu_rest[menu_rest['name_x'] == 'Bottled Water']['price'].max()
max_price_water

5.0

In [68]:
##min
min_price_water = menu_rest[menu_rest['name_x'] == 'Bottled Water']['price'].min()
min_price_water

0.0

In [69]:
bottled_water = menu_rest.loc[menu_rest['name_x'] == 'Bottled Water', ['restaurant_id', 'price', 'price_range']]
mean_price = round(bottled_water.price.mean(),2)
mean_price

2.4

In [48]:
counts = menu_rest[(menu_rest['name_x'] == 'Bottled Water') & (menu_rest['price'] < 2.4)].groupby('price_range').size()

print(counts)

price_range
$       528
$$       39
$$$$      1
dtype: int64


In [49]:
counts2 = menu_rest[(menu_rest['name_x'] == 'Bottled Water') & (menu_rest['price'] > 2.4)].groupby('price_range').size()

print(counts2)

price_range
$      597
$$      69
$$$      1
dtype: int64


In [43]:
##ile restauracji ma cene woy poniżej i powyzej średniej
num_powyzej = (bottled_water['price'] > 2.3).value_counts()[True]
num_ponizej = (bottled_water['price'] <= 2.3).value_counts()[True]

num_ponizej

536

In [44]:
num_powyzej 

852

In [45]:
counts = menu_rest[(menu_rest['name_x'] == 'Bottled Water') & (menu_rest['price'] < 2.3)].groupby('price_range').size()
counts = pd.DataFrame(counts, columns=['count'])

# zliczamy pozycje dla Iced Tea z ceną < 2.59
counts2 = menu_rest[(menu_rest['name_x'] == 'Bottled Water') & (menu_rest['price'] > 2.3)].groupby('price_range').size()
counts2 = pd.DataFrame(counts2, columns=['count2'])

# łączymy obie serie w jedną tabelę
counts_water = counts.join(counts2, how='outer')

print(counts_water)

             count  count2
price_range               
$            393.0   745.0
$$            35.0    70.0
$$$            NaN     1.0
$$$$           1.0     NaN


In [ ]:
###Iced Tea'

In [46]:
iced_tea = menu.loc[menu['name'] == 'Iced Tea', ['restaurant_id', 'price']]
mean_price = round(iced_tea.price.mean(),2)
mean_price

2.59

In [ ]:
count = menu[(menu['name'] == 'Iced Tea') & (menu['price'] > 2.59)]['name'].count()
count2 = menu[(menu['name'] == 'Iced Tea') & (menu['price'] < 2.59)]['name'].count()
count
count2

In [ ]:
max_price_icedtea = menu_rest[menu_rest['name_x'] == 'Iced Tea']['price'].max()

min_price_icedtea = menu_rest[menu_rest['name_x'] == 'Iced Tea']['price'].min()
max_price_icedtea

In [ ]:
counts_tea = menu_rest[(menu_rest['name_x'] == 'Iced Tea') & (menu_rest['price'] < 2.59)].groupby('price_range').size()
counts_tea

In [ ]:
counts_tea2 = menu_rest[(menu_rest['name_x'] == 'Iced Tea') & (menu_rest['price'] > 2.59)].groupby('price_range').size()
counts_tea2

In [ ]:


# zliczamy pozycje dla Iced Tea z ceną > 2.59
counts_tea2 = menu_rest[(menu_rest['name_x'] == 'Iced Tea') & (menu_rest['price'] > 2.59)].groupby('price_range').size()
counts_tea2 = pd.DataFrame(counts_tea2, columns=['tea>2,59'])

# zliczamy pozycje dla Iced Tea z ceną < 2.59
counts_tea = menu_rest[(menu_rest['name_x'] == 'Iced Tea') & (menu_rest['price'] < 2.59)].groupby('price_range').size()
counts_tea = pd.DataFrame(counts_tea, columns=['tea<2,59'])

# łączymy obie serie w jedną tabelę
counts_tea = counts_tea.join(counts_tea2, how='outer')

print(counts_tea)


In [ ]:
##sprite

In [ ]:
sprite = menu.loc[menu['name'] == 'Sprite', ['restaurant_id', 'price']]
mean_price = round(sprite.price.mean(),2)
mean_price

In [ ]:
count = menu[(menu['name'] == 'Sprite') & (menu['price'] > 2.63)]['name'].count()
count2 = menu[(menu['name'] == 'Sprite') & (menu['price'] < 2.63)]['name'].count()
count

In [ ]:
max_price_sprite = menu_rest[menu_rest['name_x'] == 'Sprite']['price'].max()

min_price_sprite = menu_rest[menu_rest['name_x'] == 'Sprite']['price'].min()
max_price_sprite

In [ ]:
sprite = menu_rest[(menu_rest['name_x'] == 'Sprite') & (menu_rest['price'] > 2.63)].groupby('price_range').size()
sprite = pd.DataFrame(sprite, columns=['sprite'])

# zliczamy pozycje dla Iced Tea z ceną < 2.59
sprite2 = menu_rest[(menu_rest['name_x'] == 'Sprite') & (menu_rest['price'] < 2.63)].groupby('price_range').size()
sprite2 = pd.DataFrame(sprite2, columns=['sprite2'])

# łączymy obie serie w jedną tabelę
counts_sprite = sprite.join(sprite2, how='outer')

print(counts_sprite)

In [ ]:
##milk

In [ ]:
milk = menu.loc[menu['name'] == 'Milk', ['restaurant_id', 'price']]
mean_price = round(milk.price.mean(),2)
mean_price

In [ ]:
count = menu[(menu['name'] == 'Milk') & (menu['price'] > 2.44)]['name'].count()
count2 = menu[(menu['name'] == 'Milk') & (menu['price'] < 2.44)]['name'].count()
count

In [ ]:
milk = menu_rest[(menu_rest['name_x'] == 'Milk') & (menu_rest['price'] > 2.44)].groupby('price_range').size()
milk = pd.DataFrame(milk, columns=['milk > 2,44'])

# zliczamy pozycje dla Iced Tea z ceną < 2.59
milk2 = menu_rest[(menu_rest['name_x'] == 'Milk') & (menu_rest['price'] < 2.44)].groupby('price_range').size()
milk2 = pd.DataFrame(milk2, columns=['milk <2,44'])

# łączymy obie serie w jedną tabelę
counts_milk = milk.join(milk2, how='outer')

print(counts_milk)

In [ ]:
##lemonade

In [ ]:
lemonade = menu.loc[menu['name'] == 'Lemonade', ['restaurant_id', 'price']]
mean_price = round(lemonade.price.mean(),2)
mean_price

In [ ]:
lemonade = menu_rest[(menu_rest['name_x'] == 'Lemonade') & (menu_rest['price'] > 3.69)].groupby('price_range').size()
lemonade = pd.DataFrame(lemonade, columns=['lemonade > 3,69'])

# zliczamy pozycje dla Iced Tea z ceną < 2.59
lemonade2 = menu_rest[(menu_rest['name_x'] == 'Lemonade') & (menu_rest['price'] < 3.69)].groupby('price_range').size()
lemonade2 = pd.DataFrame(lemonade2, columns=['lemonade < 3,69'])

# łączymy obie serie w jedną tabelę
counts_lemonade = lemonade.join(lemonade2, how='outer')

print(counts_lemonade)

In [ ]:
## diet coke

In [ ]:
diet_coke = menu.loc[menu['name'] == 'Diet Coke', ['restaurant_id', 'price']]
mean_price = round(diet_coke.price.mean(),2)
mean_price

In [ ]:
diet_coke = menu_rest[(menu_rest['name_x'] == 'Diet Coke') & (menu_rest['price'] > 2.64)].groupby('price_range').size()
diet_coke = pd.DataFrame(diet_coke, columns=['diet_coke > 2,64'])

# zliczamy pozycje dla Iced Tea z ceną < 2.59
diet_coke2 = menu_rest[(menu_rest['name_x'] == 'Diet Coke') & (menu_rest['price'] < 2.64)].groupby('price_range').size()
diet_coke2 = pd.DataFrame(diet_coke2, columns=['diet_coke2 < 2,64'])

# łączymy obie serie w jedną tabelę
counts_diet_coke = diet_coke.join(diet_coke2, how='outer')

print(counts_diet_coke)

In [57]:
menu_rest.price_range.value_counts()

1.0    1794458
2.0     425377
3.0       6785
4.0        672
Name: price_range, dtype: int64

In [55]:


# zamiana wartości w kolumnie 'Price_Rank' zgodnie z zasadą $ -> 1, $$ -> 2, $$$ -> 3, $$$$ -> 4
menu_rest['price_range'] = menu_rest['price_range'].replace({'$': 1, '$$': 2, '$$$': 3, '$$$$': 4})


In [58]:
cor_water = menu_rest[menu_rest['name_x'] == 'Bottled Water']
correlation = menu_rest['price'].corr(cor_water['price_range'])

print(correlation)

##wynik wskazuje, że droga restauracja NIE liczy więcej za wodę.

0.050976387721716566


In [59]:
cor_iced_tea = menu_rest[menu_rest['name_x'] == 'Iced Tea']
correlation1 = menu_rest['price'].corr(cor_iced_tea['price_range'])

print(correlation1)

0.15935127930917012


In [60]:
cor_sprite = menu_rest[menu_rest['name_x'] == 'Sprite']
correlation2 = menu_rest['price'].corr(cor_sprite['price_range'])

print(correlation2)

0.06560712664283648


In [61]:
cor_milk = menu_rest[menu_rest['name_x'] == 'Milk']
correlation3 = menu_rest['price'].corr(cor_milk['price_range'])

print(correlation3)

0.38369213116715256


In [62]:
cor_lem = menu_rest[menu_rest['name_x'] == 'Lemonade']
correlation4 = menu_rest['price'].corr(cor_lem['price_range'])

print(correlation4)

0.6411946580957548


In [63]:
cor_coke = menu_rest[menu_rest['name_x'] == 'Diet Coke']
correlation5 = menu_rest['price'].corr(cor_coke['price_range'])

print(correlation5)

0.025757147333739395


In [ ]:
#powyższe wskazuje, że w przypadku wody w butelce rank cenowy restauracji nie ma wpywu na cenę wody, ale dla mleka, 
##lemoniady czy ice tea ten wskaźnik rośnie. Co wskazuje, że im wyższy rank tym cena jest wyższa za te naoje.

In [ ]:
##Można powiedzieć, że istnieje umiarkowany dodatni związek między rankiem restauracji a ceną napoju, 
##ale nie można na tej podstawie wnioskować, że rank restauracji ma bezpośredni wpływ na cenę napoju. 
##Wzrost rankingu restauracji może być po prostu skorelowany z wyższymi cenami napojów, 
##ale przyczyną tego związku może być na przykład poziom ekskluzywności restauracji lub 
##średnie zarobki klientów, którzy odwiedzają tę restaurację.

In [ ]:
###pizza

In [ ]:
# wyszukiwanie wartości w kolumnie 'category', które zawierają słowo 'pizza'
pattern = re.compile(r'\b\w*Pizza\w*\b', flags=re.IGNORECASE) # wyrażenie regularne dla słowa 'pizza'
pizza_rows = menu_rest[menu_rest['category_x'].str.contains(pattern, regex=True)]
pizza_categories = pizza_rows['category_x'].unique()


In [ ]:
pizza_counts = menu_rest[menu_rest['category_x'].str.contains('Pizza', case=False)]['category_x'].value_counts()

pizza_count

In [ ]:
pizza_category = menu_rest[menu_rest['category_x'].str.contains('Pizza', case=False, regex=True)]['category_x'].value_counts()
pizza_category[:10]

In [ ]:
cor_pizza = menu_rest[menu_rest['category_x'] == 'Pizza']
correlation_p1 = menu_rest['price'].corr(cor_pizza['price_range'])

print(correlation_p1)

In [ ]:
cor_pizza2 = menu_rest[menu_rest['category_x'] == 'Specialty Pizzas']
correlation_p2 = menu_rest['price'].corr(cor_pizza2['price_range'])

print(correlation_p2)

In [ ]:

pizza3 = menu_rest.loc[menu_rest['category_x'] == 'Pizzas']
corr3 = pizza3['price_range'].corr(pizza3['price'])
print(corr3)

In [ ]:

pizza4 = menu_rest.loc[menu_rest['category_x'] == 'Specialty Pizza']
corr4 = pizza4['price_range'].corr(pizza4['price'])

print(corr4)

In [ ]:

pizza5 = menu_rest.loc[menu_rest['category_x'] == 'Large Specialty Pizza']
corr = pizza5['price_range'].corr(pizza5['price'])

print(corr)


In [ ]:
pizza6 = menu_rest.loc[menu_rest['category_x'] == 'Create Your Own Pizza']
corr6 = pizza6['price_range'].corr(pizza6['price'])

print(corr6)

In [ ]:
unique_categories = menu_rest['category_y'].nunique()
unique_categories

In [ ]:
category_counts = menu_rest['category_y'].value_counts()
category_counts